# PRÁCTICA 6. BASADOS EN INSTANCIAS
PATRICIA AGUADO LABRADOR

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score as ac

Cargar archivo de datos con extensión csv

In [2]:
file = pd.read_csv('digits_from_5_to_9.csv')
file

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,digit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4853,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4854,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [3]:
data = np.asarray(file)
datasets1 = []
datasets2 = []

Función para seleccionar las parejas de dígitos que mejor se difencian del 5 al 9 con el clasificador k vecinos más próximos (variando k de 1 a 7)

In [4]:
''' 
entrada: array formado por 5 conjuntos de datos, cada uno tiene por clase un único dígito (del 5 al 9)

salida: array con los dígitos que más diferencias presentan y la tasa de acierto del clasificador KNN
        para cada valor de k
'''

def best_different_digits (datasets):
    # Almacenará la pareja de dígitos que mejor se diferencian para cada valor de k
    result = []
    
    # Para cada valor de k (vecinos) creamos el clasificador para todas las combinaciones 
    # sin repetición de dígitos
    for neighbors in range(1,8):
        digits_couple = []
        best_acu = 0
        best_couple = []
        
        for set1 in range(len(datasets)):
            for set2 in range(set1+1,len(datasets)):
                # Combinamos dos conjuntos de datos de diferente clase
                data = np.concatenate((datasets[set1], datasets[set2]), axis = 0)
                
                digits = np.unique(data[:,-1])
                
                # Separamos los valores de atributos de los valores de la clase
                X = data[:,:784]
                y = data[:,-1]
                
                # Separamos el conjunto de datos en 2/3 para entrenamiento y 1/3 para prueba
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)
                
                # Creamos y entrenamos el clasificador
                knn = KNeighborsClassifier(n_neighbors = neighbors, algorithm ='brute')
                knn_fit = knn.fit(X_train, y_train)
                predicted = knn_fit.predict(X_test)

                acu = ac(predicted, y_test)
                
                # Diferentes parejas de dígitos con su tasa de acierto para un valor de k
                digits_couple.append([digits[0], digits[1], acu])

        # Calcular la pareja de dígitos que mejor se distinguen para un valor de k, tomando
        # el conjunto de dígitos que mayor tasa de acierto tuvo        
        for i in range(len(digits_couple)):
            if (digits_couple[i][-1] > best_acu):
                best_acu = digits_couple[i][-1]
                best_couple = [digits_couple[i][0],digits_couple[i][1]]

        result.append([best_couple[0], best_couple[1], best_acu])

    return result

Función para seleccionar el mejor valor de k del clasificador KNN para una pareja de dígitos

In [5]:
''' 
entrada: array formado por parejas de dígitos que mejor se diferencian para cada valor diferente
         con el que se creó el clasificador KNN

salida: array con el mejor valor de k y su tasa de acierto para una pareja de dígitos dada
'''

def k_optimal_per_digit(results):
    c = 0
    
    # Sacamos las parejas de dígitos diferentes que hay en el array
    diff_digits = np.unique(results[:,:2],axis=0, return_counts=True)
    k_optimal = np.zeros((len(diff_digits[0]),4),dtype='float')

    for digit in diff_digits[0]:
        # Cambiamos el tipo de la pareja de dígitos a int para poder hacer una comparación
        digit = list(map(int,digit))
        acu = 0
        k = 0

        for j in range(len(results[:,:2])):
            digit1 = list(map(int,results[:,:2][j]))

            if (digit == digit1):
                # Seleccionamos la mejor tasa de error y el mejor k para cada pareja de dígitos
                # del array de entrada
                if (results[j][2]>acu):
                    acu = results[j][2]
                    k = j+1

        k_optimal[c] = [digit[0], digit[1], acu, k]
        c += 1

    return k_optimal

## PARTE I

In [6]:
diff_digits = np.unique(data[:,-1])

# Separamos el archivo en diferentes conjuntos de datos en función de los valores que puede tomar la clase
for digit in range(diff_digits.shape[0]):
    index_digit = np.where(data[:,-1] == diff_digits[digit])
    digit_data = data[index_digit]
    
    datasets1.append(digit_data)
    
# Aplicamos KNN
results = best_different_digits(datasets1)

# Resultados    
for k in range(7):
    print("\033[1mk =",k+1,"\033[0m",
            "\n  Dígitos que mejor se diferencian: ", results[k][0], " y ", results[k][1],
            "\n  Tasa de acierto: ",results[k][2],"\n")

k = 1  
  Dígitos que mejor se diferencian:  6  y  7 
  Tasa de acierto:  1.0 

k = 2  
  Dígitos que mejor se diferencian:  6  y  7 
  Tasa de acierto:  1.0 

k = 3  
  Dígitos que mejor se diferencian:  5  y  7 
  Tasa de acierto:  1.0 

k = 4  
  Dígitos que mejor se diferencian:  6  y  7 
  Tasa de acierto:  1.0 

k = 5  
  Dígitos que mejor se diferencian:  6  y  9 
  Tasa de acierto:  1.0 

k = 6  
  Dígitos que mejor se diferencian:  5  y  7 
  Tasa de acierto:  1.0 

k = 7  
  Dígitos que mejor se diferencian:  6  y  7 
  Tasa de acierto:  0.9969788519637462 



In [7]:
results = np.asarray(results)

# Buscamos el k óptimo
k_optimal = k_optimal_per_digit(results)

print("\033[1mK ÓPTIMO PARA CADA PAREJA DE DÍGITOS:\033[0m\n")

# Resultados
for i in range(k_optimal.shape[0]):
    print("   Para los dígitos:", int(k_optimal[i][0]),"y", int(k_optimal[i][1]),
            ", el \33[4mk óptimo es", int(k_optimal[i][3]),"\33[0m",
            "con una tasa de acierto de",k_optimal[i][2],"\n")

K ÓPTIMO PARA CADA PAREJA DE DÍGITOS:

   Para los dígitos: 5 y 7 , el k óptimo es 3  con una tasa de acierto de 1.0 

   Para los dígitos: 6 y 7 , el k óptimo es 1  con una tasa de acierto de 1.0 

   Para los dígitos: 6 y 9 , el k óptimo es 5  con una tasa de acierto de 1.0 



## PARTE II

In [8]:
atributes = data[:,:784]
target = data[:,-1].reshape(4856,1)

# Reducimos los píxeles a bits, cambiando a 1 el valor de los atributos distintos de cero
pixel_diff_zero = np.nonzero(atributes)
atributes[pixel_diff_zero] = 1
data1 = np.concatenate((atributes, target), axis =1)

diff_digits = np.unique(data1[:,-1])

# Separamos el archivo en diferentes conjuntos de datos en función de los valores que puede tomar la clase
for digit in range(diff_digits.shape[0]):
    index_digit = np.where(data1[:,-1] == diff_digits[digit])
    digit_data = data[index_digit]
    
    datasets2.append(digit_data)
    
# Aplicamos KNN    
results = best_different_digits(datasets2)

# Resultados
for k in range(7):
    print("\033[1mk =",k+1,"\033[0m",
          "\n  Dígitos que mejor se diferencian:", results[k][0], "y", results[k][1],
          "\n  Tasa de acierto:",results[k][2],"\n")

k = 1  
  Dígitos que mejor se diferencian: 6 y 7 
  Tasa de acierto: 1.0 

k = 2  
  Dígitos que mejor se diferencian: 6 y 9 
  Tasa de acierto: 1.0 

k = 3  
  Dígitos que mejor se diferencian: 6 y 9 
  Tasa de acierto: 1.0 

k = 4  
  Dígitos que mejor se diferencian: 5 y 6 
  Tasa de acierto: 1.0 

k = 5  
  Dígitos que mejor se diferencian: 6 y 7 
  Tasa de acierto: 0.9969788519637462 

k = 6  
  Dígitos que mejor se diferencian: 6 y 7 
  Tasa de acierto: 1.0 

k = 7  
  Dígitos que mejor se diferencian: 6 y 7 
  Tasa de acierto: 0.9984894259818731 



In [9]:
results = np.asarray(results)

# Buscamos el k óptimo
k_optimal = k_optimal_per_digit(results)

print("\033[1mK ÓPTIMO PARA CADA PAREJA DE DÍGITOS:\033[0m\n")

# Resultados
for i in range(k_optimal.shape[0]):
    print("   Para los dígitos:", int(k_optimal[i][0]),"y", int(k_optimal[i][1]),
            ", el \33[4mk óptimo es", int(k_optimal[i][3]),"\33[0m",
            "con una tasa de acierto de",k_optimal[i][2],"\n")

K ÓPTIMO PARA CADA PAREJA DE DÍGITOS:

   Para los dígitos: 5 y 6 , el k óptimo es 4  con una tasa de acierto de 1.0 

   Para los dígitos: 6 y 7 , el k óptimo es 1  con una tasa de acierto de 1.0 

   Para los dígitos: 6 y 9 , el k óptimo es 2  con una tasa de acierto de 1.0 

